In [7]:
# Setup and Imports
%load_ext autoreload
%autoreload 2

import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from uncertainties import ufloat
from matplotlib.lines import Line2D
from matplotlib.patches import Patch

# Add the src directory to import snowpyt_mechparams
sys.path.append('../src')
from snowpilot_utils import convert_grain_form, parse_sample_pits
from snowpyt_mechparams import density, elastic_modulus, shear_modulus, poissons_ratio


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Parse all snowpit files from the data folder
all_pits = parse_sample_pits('data')


Successfully parsed 50278 files
Failed to parse 0 files


In [3]:
# Calculate densities for all layers using kim_geldsetzer method
layer_info = []

for pit in all_pits:
    for layer in pit.snow_profile.layers:
        layer_dict = {
            'pit_id': pit.core_info.pit_id,
            'hand_hardness': layer.hardness,
            'depth_top': layer.depth_top[0] if layer.depth_top else None,  
            'thickness': layer.thickness[0] if layer.thickness else None,
        }

        # Convert grain form and calculate density
        grain_form = convert_grain_form(layer.grain_form_primary, 'kim_geldsetzer') if layer.grain_form_primary else None
        layer_dict['kim_geldsetzer_grain_form'] = grain_form
        
        # Calculate density if we have valid inputs
        if layer.hardness and grain_form:
            try:
                density_ufloat = density.calculate_density( 
                    method='kim_geldsetzer',
                    hand_hardness=layer.hardness,
                    grain_form=grain_form
                )
                layer_dict['density_kim_geldsetzer'] = density_ufloat.nominal_value
                layer_dict['density_kim_geldsetzer_uncertainty'] = density_ufloat.std_dev
            except Exception:
                layer_dict['density_kim_geldsetzer'] = None
                layer_dict['density_kim_geldsetzer_uncertainty'] = None
        else:
            layer_dict['density_kim_geldsetzer'] = None
            layer_dict['density_kim_geldsetzer_uncertainty'] = None

        layer_info.append(layer_dict)

# Create dataframe
layer_df = pd.DataFrame(layer_info)


In [ ]:
# Calculate elastic modulus and shear modulus using wautier method

# Function to calculate elastic modulus and shear modulus for a single row
def calculate_modulii_row(row):
    result = {'e_mod_wautier': None, 's_mod_wautier': None}
    
    # Skip if no grain form or density data
    if pd.isna(row['kim_geldsetzer_grain_form']) or pd.isna(row['density_kim_geldsetzer']):
        return pd.Series(result)
    
    try:
        # Create density ufloat once
        density_ufloat = ufloat(row['density_kim_geldsetzer'], row['density_kim_geldsetzer_uncertainty'])
        grain_form = row['kim_geldsetzer_grain_form']
        
        # Calculate elastic modulus using wautier method
        try:
            e_mod = elastic_modulus.calculate_elastic_modulus(
                method='wautier',
                density=density_ufloat,
                grain_form=grain_form
            )
            result['e_mod_wautier'] = e_mod.nominal_value
        except Exception:
            pass
        
        # Calculate shear modulus using wautier method
        try:
            s_mod = shear_modulus.calculate_shear_modulus(
                method='wautier',
                density=density_ufloat,
                grain_form=grain_form
            )
            result['s_mod_wautier'] = s_mod.nominal_value
        except Exception:
            pass
    except Exception:
        pass
    
    return pd.Series(result)

# Apply the function to all rows at once
layer_df[['e_mod_wautier', 's_mod_wautier']] = layer_df.apply(calculate_modulii_row, axis=1)


/Users/marykateconnelly/Desktop/F25/SnowPyt-MechParams/venv/lib/python3.13/site-packages/uncertainties/core.py:1024: UserWarning: Using UFloat objects with std_dev==0 may give unexpected results.
  warn("Using UFloat objects with std_dev==0 may give unexpected results.")


In [8]:
# Function to calculate poissons ratio for a single row using multiple methods
def calculate_poissons_ratio_row(row):
    """Calculate poissons ratio for a single row using multiple methods."""
    result = {
        'nu_kochle': np.nan,
        'nu_srivastava': np.nan,
        'nu_from_modulii': np.nan
    }
    
    # Skip if no grain form or density data
    if pd.isna(row['kim_geldsetzer_grain_form']) or pd.isna(row['density_kim_geldsetzer']):
        return pd.Series(result)
    
    try:
        # Create density ufloat once
        density_ufloat = ufloat(row['density_kim_geldsetzer'], row['density_kim_geldsetzer_uncertainty'])
        grain_form = row['kim_geldsetzer_grain_form']
        
        # Calculate Poisson's ratio using Kochle method
        try:
            nu_kochle = poissons_ratio.calculate_poissons_ratio(
                method='kochle',
                grain_form=grain_form
            )
            result['nu_kochle'] = nu_kochle.nominal_value
        except Exception:
            pass
        
        # Calculate Poisson's ratio using Srivastava method
        try:
            nu_srivastava = poissons_ratio.calculate_poissons_ratio(
                method='srivastava',
                density=density_ufloat,
                grain_form=grain_form
            )
            result['nu_srivastava'] = nu_srivastava.nominal_value
        except Exception:
            pass
        
        # Calculate Poisson's ratio from modulii (using wautier E and G)
        # Only calculate if both e_mod_wautier and s_mod_wautier are available
        if not pd.isna(row['e_mod_wautier']) and not pd.isna(row['s_mod_wautier']):
            try:
                # Create ufloats for the moduli (assuming no uncertainty stored)
                e_mod_ufloat = ufloat(row['e_mod_wautier'], 0)
                s_mod_ufloat = ufloat(row['s_mod_wautier'], 0)
                
                nu_from_modulii = poissons_ratio.calculate_poissons_ratio(
                    method='from_modulii',
                    elastic_modulus=e_mod_ufloat,
                    shear_modulus=s_mod_ufloat
                )
                result['nu_from_modulii'] = nu_from_modulii.nominal_value
            except Exception:
                pass
        
    except Exception:
        pass
    
    return pd.Series(result)

# Apply the function to all rows at once
layer_df[['nu_kochle', 'nu_srivastava', 'nu_from_modulii']] = layer_df.apply(calculate_poissons_ratio_row, axis=1)


/Users/marykateconnelly/Desktop/F25/SnowPyt-MechParams/venv/lib/python3.13/site-packages/uncertainties/core.py:1024: UserWarning: Using UFloat objects with std_dev==0 may give unexpected results.
  warn("Using UFloat objects with std_dev==0 may give unexpected results.")
